In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import csv
import datetime

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #ez hogy a rákban nem a defaultja a jupyternek?

# saját függvényeink
from functions import *

In [2]:
bipartite_data = createNxGraph()
projected_graph = listMEPs()
mep_social_network = nx.algorithms.bipartite.projected_graph(bipartite_data, projected_graph)

# SPLITTING THE DATA

In [3]:
import datetime

In [4]:
ENTIRE_DATASET["Date"] = pd.to_datetime(ENTIRE_DATASET["Date"], format = "%d/%m/%Y")

In [6]:
# https://stackoverflow.com/questions/55268578/split-dataframe-by-month

# groupby your key and freq
g = ENTIRE_DATASET.groupby(pd.Grouper(key='Date', freq='M'))
# groups to a list of dataframes with list comprehension
split_data = [group for _,group in g]

In [7]:
s = 0
for i in split_data:
    x = len(i)
    print(x)
    s += x
print(f"SUM = {s}")

229
397
1935
3730
7928
3932
6730
9869
6034
8081
40262
31372
27835
929
39157
30400
16180
23771
11720
31572
23648
23850
20243
42070
32784
223
18193
19887
13349
35827
19644
54861
48387
20055
13035
39045
19659
3956
193
SUM = 750972


^- Ennyi adatpontunk van intervallumonként. Nekem balanceoltnak tűnik őszintén szólva.

Most megcsináljuk basically ugyanazt az analízist mint fönt, csak minden intervallumra külön.

In [8]:
def listMEPs_div(GDataset): # listing the different MEP names for the given dataset
    o = pd.unique(GDataset['MEPName'])
    return o
ListsMEP = [set(listMEPs_div(miniGraph)) for miniGraph in split_data]#list for each subgraph
#print(ListsMEP)
meps_who_did_something_in_all_intervals = set.intersection(*ListsMEP)
print(meps_who_did_something_in_all_intervals)
print(len(meps_who_did_something_in_all_intervals))

set()
0


In [9]:
by_country = {}
by_polgroup = {}
for mep in meps_who_did_something_in_all_intervals:
    curr_country = getMEPData("MEPName", mep)["MemberState"]
    curr_polgroup = getMEPData("MEPName", mep)["EPGroup"]
    if curr_country not in by_country:
        by_country[curr_country] = 0
    if curr_polgroup not in by_polgroup:
        by_polgroup[curr_polgroup] = 0
    by_country[curr_country] += 1
    by_polgroup[curr_polgroup] += 1

print(by_country)
print(by_polgroup)
    

{}
{}


In [10]:
for i in split_data:
    contribs_by_country = {}
    for node in mep_social_network:
        curr_country = getMEPData("MEPName", node)["MemberState"]
        if curr_country not in contribs_by_country:
            contribs_by_country[curr_country] = 0
        contribs_by_country[curr_country] += mep_social_network.degree(node)

KeyboardInterrupt: 